# Project Title
### Data Engineering Capstone Project

#### Project Summary
This project will build up a data warehouse as a single-source-of-truth database by combining four data sets containing immigration data, airport codes, demographics of US cities and global temperature data. 

For example, link between the average temperature and the US immigration US state, and what the current dempgraphic layout of each US stat?

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [57]:
import pandas as pd
import configparser
import os
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, count, sum, mean, round, upper, lower, isnan
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, monotonically_increasing_id
from pyspark.sql.types import *
from pyspark.sql.types import DoubleType, StringType, IntegerType, FloatType, DateType
from pyspark.sql import functions as F

#Import ETL functions
import etl_functions as ef
#from sql_queries import analytics_queries

In [58]:
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

os.environ["AWS_ACCESS_KEY_ID"] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"] = config['AWS']['AWS_SECRET_ACCESS_KEY']

In [59]:
#spark = SparkSession.builder.\
#config("spark.jars.repositories", "https://repos.spark-packages.org/").\
#config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
#enableHiveSupport().getOrCreate()

#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [60]:
#write to parquet
#df_spark.write.parquet("sas_data")
#df_spark=spark.read.parquet("sas_data")
#df_spark.show(3)

###### Below SparkSession configurations should be done to be able to write into S3 Buckets (according to the Spark version over the server)

In [61]:
spark = SparkSession.builder\
                    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")\
                    .enableHiveSupport().getOrCreate()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

In this project we will gather the data from four differnt data sources. here you are the steps that will be followed:
1. Read and load into staging Dataframes
2. Apply cleansing over all those DFs
3. Perform some ETL trasformation rules using pyspark
4. Model them into Fact & Dimention tables using start schema
5. Write those DFs into S3 buckets
6. Extract again and apply Data Quality checks

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

- i94 Immigration Sample Data: Sample data of immigration records from the US National Tourism and Trade Office. This is the main data source that will serve as the Fact table in the schema. This data comes from https://travel.trade.gov/research/reports/i94/historical/2016.html.
- World Temperature Data world_temperature. This dataset contains temperature data in various cities from the 1700’s to 2013. Although the data is only recorded until 2013, we can use this as an average/gauge of temperature in 2017. This data comes from https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data.
- US City Demographic Data: Data about the demographics of US cities. This dataset includes information on the population of all US cities such as race, household size and gender. This data comes from https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/.
- Airport Codes: This table contains the airport codes for the airports in corresponding cities. This data comes from https://datahub.io/core/airport-codes#data.


##### TEMPERATURE DATA

In [6]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
temperature_df_orig = pd.read_csv(fname)

##### Due to performance, I'll restrict over the Temprature readings in USA

In [7]:
temperature_df_orig=temperature_df_orig.dropna()
temperature_df_orig.head() #8235082

temperature_df_orig.groupby(['Country'])['dt'].count().sort_values(ascending=False).head(20)

Country
India             960548
China             819132
United States     661524
Brazil            451645
Russia            448663
Japan             356467
Indonesia         273295
Germany           256446
United Kingdom    215288
Mexico            208020
Nigeria           157012
Spain             156528
Turkey            147086
Iran              146168
Italy             132972
Philippines       124865
Pakistan          123265
Poland            120308
France            113976
South Africa       93653
Name: dt, dtype: int64

In [8]:
temperature_df = temperature_df_orig[temperature_df_orig['Country'] == 'United States']
#temperature_df=temperature_df_orig

##### AIRPORT CODES

In [9]:
airport_codes = 'airport-codes_csv.csv'
airport_df = pd.read_csv(airport_codes)

In [10]:
airport_df.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


##### IMMIGRATION DATA

In [11]:
immigration_data = 'immigration_data_sample.csv'
immigration_df = pd.read_csv(immigration_data)

In [12]:
immigration_df.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


##### US CITIES DEMOGRAPHICS

In [13]:
us_cities_demographics = 'us-cities-demographics.csv'
demographics_df = spark.read.csv(us_cities_demographics, inferSchema=True, header=True, sep=';')

In [14]:
demographics_df.limit(5).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data:
- Drop rows containing all values with NULL
- Drop duplicate values


In [15]:
# Performing cleaning tasks here
# Delete those missing all values rows and then duplicates in all DFs
# Temperature DF
temperature_clean = temperature_df.dropna(how='all')
temperature_clean = temperature_clean.drop_duplicates()

# Airport DF
airport_clean = airport_df.dropna(how='all')
airport_clean = airport_clean.drop_duplicates()

# Immigration DF
immigration_clean = immigration_df.dropna(how='all')
immigration_clean = immigration_clean.drop_duplicates()

# Demographics DF
demographics_clean = demographics_df.dropna(how='all')
demographics_clean = demographics_clean.drop_duplicates()

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
According to Kimball Modelling and as per the nature of the 4 combined Data Sources. The main target of combining those 4 Data Sources is to track the immigration actions into the USA. Accordingly:

- 1. Identify the Facts:
    - Fact tables focus on the occurrences of a singular business process, and have a one-to-one relationship with the dimention tables.
    - The fact table identified in this project is:
        - immigration_fact
        
- 2. Identify the Dimensions:
    - Dimension tables provide any quantitative or metric around the fact event or business process.
    - The dimensions identified in this project are:
        - migrant_dim
        - status_dim
        - visa_dim
        - state_dim
        - time_dim
        - airport_codes
        - temperature_dim
        - country_codes
        - country_temperature
    
The above mentioned tables will be developed in a Relational Database Management System to form a Star Schema, which can be used in Data analytics and BI insights activities.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model:
1. Read and load into staging Dataframes
2. Apply cleansing over all those DFs
3. Perform some ETL trasformation rules using pyspark
4. Model them into Fact & Dimention tables using start schema
5. Write those DFs into S3 buckets
6. Extract again and apply Data Quality checks

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [16]:
temperature_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 661524 entries, 47555 to 8439246
Data columns (total 7 columns):
dt                               661524 non-null object
AverageTemperature               661524 non-null float64
AverageTemperatureUncertainty    661524 non-null float64
City                             661524 non-null object
Country                          661524 non-null object
Latitude                         661524 non-null object
Longitude                        661524 non-null object
dtypes: float64(2), object(5)
memory usage: 40.4+ MB


In [17]:
# create schema for Temperature
temperature_schema = StructType([StructField("dt", StringType(), True)\
                          ,StructField("AverageTemperature", FloatType(), True)\
                          ,StructField("AverageTemperatureUncertainty", FloatType(), True)\
                          ,StructField("City", StringType(), True)\
                          ,StructField("Country", StringType(), True)\
                          ,StructField("Latitude", StringType(), True)\
                          ,StructField("Longitude", StringType(), True)])

temperature_spark = spark.createDataFrame(temperature_clean, schema=temperature_schema)

temperature_spark.toPandas().head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
1,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
2,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
3,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
4,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W


In [18]:
airport_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.5+ MB


In [19]:
# create schema for Airport codes
airport_codes_schema = StructType([StructField("ident", StringType(), True)\
                        ,StructField("type", StringType(), True)\
                        ,StructField("name", StringType(), True)\
                        ,StructField("elevation_ft", FloatType(), True)\
                        ,StructField("continent", StringType(), True)\
                        ,StructField("iso_country", StringType(), True)\
                        ,StructField("iso_region", StringType(), True)\
                        ,StructField("municipality", StringType(), True)\
                        ,StructField("gps_code", StringType(), True)\
                        ,StructField("iata_code", StringType(), True)\
                        ,StructField("local_code", StringType(), True)\
                        ,StructField("coordinates", StringType(), True)])

airport_codes_spark = spark.createDataFrame(airport_clean, schema=airport_codes_schema)

airport_codes_spark.toPandas().head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [20]:
immigration_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 29 columns):
Unnamed: 0    1000 non-null int64
cicid         1000 non-null float64
i94yr         1000 non-null float64
i94mon        1000 non-null float64
i94cit        1000 non-null float64
i94res        1000 non-null float64
i94port       1000 non-null object
arrdate       1000 non-null float64
i94mode       1000 non-null float64
i94addr       941 non-null object
depdate       951 non-null float64
i94bir        1000 non-null float64
i94visa       1000 non-null float64
count         1000 non-null float64
dtadfile      1000 non-null int64
visapost      382 non-null object
occup         4 non-null object
entdepa       1000 non-null object
entdepd       954 non-null object
entdepu       0 non-null float64
matflag       954 non-null object
biryear       1000 non-null float64
dtaddto       1000 non-null object
gender        859 non-null object
insnum        35 non-null float64
airline       967 non

In [21]:
# create schema for immigration DF
immigration_schema = StructType([StructField("0", IntegerType(), True)\
                          ,StructField("cicid", FloatType(), True)\
                          ,StructField("i94yr", FloatType(), True)\
                          ,StructField("i94mon", FloatType(), True)\
                          ,StructField("i94cit", FloatType(), True)\
                          ,StructField("i94res", FloatType(), True)\
                          ,StructField("i94port", StringType(), True)\
                          ,StructField("arrdate", FloatType(), True)\
                          ,StructField("i94mode", FloatType(), True)\
                          ,StructField("i94addr", StringType(), True)\
                          ,StructField("depdate", FloatType(), True)\
                          ,StructField("i94bir", FloatType(), True)\
                          ,StructField("i94visa", FloatType(), True)\
                          ,StructField("count", FloatType(), True)\
                          ,StructField("dtadfile", StringType(), True)\
                          ,StructField("visapost", StringType(), True)\
                          ,StructField("occup", StringType(), True)\
                          ,StructField("entdepa", StringType(), True)\
                          ,StructField("entdepd", StringType(), True)\
                          ,StructField("entdepu", FloatType(), True)\
                          ,StructField("matflag", StringType(), True)\
                          ,StructField("biryear", FloatType(), True)\
                          ,StructField("dtaddto", StringType(), True)\
                          ,StructField("gender", StringType(), True)\
                          ,StructField("insnum", FloatType(), True)\
                          ,StructField("airline", StringType(), True)\
                          ,StructField("admnum", FloatType(), True)\
                          ,StructField("fltno", StringType(), True)\
                          ,StructField("visatype", StringType(), True)])

immigration_spark = spark.createDataFrame(immigration_clean, schema=immigration_schema)

immigration_spark.toPandas().head()

,0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658268e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436199e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [22]:
demographics_clean.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int32
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int32
dtypes: float64(6), int32(2), object(4)
memory usage: 248.5+ KB


In [23]:
# create schema for demographic
demographics_schema = StructType([StructField("City", StringType(), True)\
                        ,StructField("State", StringType(), True)\
                        ,StructField("Median Age", FloatType(), True)\
                        ,StructField("Male Population", FloatType(), True)\
                        ,StructField("Female Population", FloatType(), True)\
                        ,StructField("Total Population", IntegerType(), True)\
                        ,StructField("Number of Veterans", FloatType(), True)\
                        ,StructField("Foreign-born", FloatType(), True)\
                        ,StructField("Average Household Size", FloatType(), True)\
                        ,StructField("State Code", StringType(), True)\
                        ,StructField("Race", StringType(), True)\
                        ,StructField("Count", IntegerType(), True)])

demographics_spark = spark.createDataFrame(demographics_clean.toPandas(), schema=demographics_schema)

demographics_spark.toPandas().head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Maple Grove,Minnesota,38.599998,31780.0,36601.0,68381,2943.0,7645.0,2.64,MN,White,59683
1,Concord,California,39.599998,62310.0,66358.0,128668,6287.0,37428.0,2.72,CA,White,92575
2,Highlands Ranch,Colorado,39.599998,49186.0,53281.0,102467,4840.0,8827.0,2.72,CO,Asian,5650
3,Asheville,North Carolina,37.900002,42100.0,46407.0,88507,4973.0,6630.0,2.18,NC,American Indian and Alaska Native,496
4,Westland,Michigan,39.900002,37742.0,44253.0,81995,4756.0,6429.0,2.41,MI,Black or African-American,16422


##### Wrinting the tables into Parquet and upload over S3 bucket

###### Migrant dimention table

In [24]:
output_path="s3a://udacity-datalake-msaied2/Project6_Capstone/migrant.parquet"
migrant_dim =  ef.migrant_dimention(immigration_spark, output_path)

Migrant dimention table writing has been completed!


In [25]:
migrant_dim.toPandas().head()

,migrant_id,birth_year,gender
0,561,1969.0,NaN
1,367,1933.0,M
2,376,1996.0,F
3,61,1972.0,F
4,404,1993.0,NaN


###### Status dimention table

In [26]:
output_path="s3a://udacity-datalake-msaied2/Project6_Capstone/status.parquet"
status_dim =  ef.status_dimension(immigration_spark, output_path)

Status dimention table writing has been completed!


In [27]:
status_dim.toPandas().head()

,status_flag_id,arrival_flag,departure_flag,match_flag
0,0,G,O,M
1,14,G,K,M
2,157,O,I,M
3,4,Z,K,M
4,15,P,D,M


###### Visa dimention table

In [28]:
output_path="s3a://udacity-datalake-msaied2/Project6_Capstone/visa.parquet"
visa_dim =  ef.visa_dimension(immigration_spark, output_path)

Visa dimention table writing has been completed!


In [29]:
visa_dim.toPandas().head()

,visa_id,i94visa,visatype,visapost
0,1,2.0,B2,MTR
1,95,1.0,B1,RDJ
2,407,2.0,B2,SYD
3,353,2.0,B2,RCF
4,671,1.0,B1,MNL


###### State dimention table

In [30]:
output_path="s3a://udacity-datalake-msaied2/Project6_Capstone/state.parquet"
state_dim =  ef.state_dimension(demographics_spark, output_path)

State dimention table writing has been completed!


In [31]:
state_dim.toPandas().head()

,state_code,state,median_age,total_population,male_population,female_population,foreign_born,average_household_size
0,MT,Montana,35.50,906470,438535.0,467935.0,29885.0,2.28
1,NC,North Carolina,33.79,15300995,7330525.0,7970470.0,1896635.0,2.48
2,MD,Maryland,36.37,6560645,3139755.0,3420890.0,1148970.0,2.65
3,CO,Colorado,35.82,14678345,7273095.0,7405250.0,1688155.0,2.56
4,CT,Connecticut,35.00,4355096,2123435.0,2231661.0,1114250.0,2.67


###### Time dimention table

In [32]:
output_path="s3a://udacity-datalake-msaied2/Project6_Capstone/time.parquet"
time_dim =  ef.time_dimension(immigration_spark, output_path)

Time dimention table writing has been completed!


In [33]:
time_dim.toPandas().head()

,arrdate,arrival_date,day,month,year,week,weekday
0,20555.0,None,None,None,None,None,None
1,20546.0,None,None,None,None,None,None
2,20550.0,None,None,None,None,None,None
3,20557.0,None,None,None,None,None,None
4,20574.0,None,None,None,None,None,None


###### Airport_Codes dimention table

In [34]:
output_path="s3a://udacity-datalake-msaied2/Project6_Capstone/airport_codes.parquet"
airport_codes =  ef.airport_codes_dimension(airport_codes_spark, output_path)

Airport_Codes dimention table writing has been completed!


In [35]:
airport_codes.toPandas().head()

,ident,type,iata_code,name,iso_country,iso_region,municipality,gps_code,coordinates,elevation_ft
0,06IN,closed,NaN,Ellis Fly-In Airport,US,US-IN,Blackhawk,NaN,"-87.303596, 39.282799",575.0
1,06VA,small_airport,NaN,Mount Horeb Field,US,US-VA,Grottoes,06VA,"-78.85530090332031, 38.249000549316406",1160.0
2,0LA0,heliport,NaN,West Hackberry Heliport,US,US-LA,Hackberry,0LA0,"-93.40019989013672, 30.008499145507812",10.0
3,0MD6,small_airport,NaN,Walters Airport,US,US-MD,Mount Airy,0MD6,"-77.10579681396484, 39.38119888305664",750.0
4,0OH7,small_airport,NaN,Apple Airport,US,US-OH,Piqua,0OH7,"-84.1718978881836, 40.1432991027832",1000.0


###### Temperature dimention table

In [36]:
output_path="s3a://udacity-datalake-msaied2/Project6_Capstone/temperature.parquet"
temperature_dim=ef.temperature_dimension(temperature_spark, output_path)

Temperature dimention table writing has been completed!


In [37]:
temperature_dim.toPandas().head()

,temperature_id,country,average_temperature,average_temperature_uncertainty
0,506806140928,United States,13.95,1.09


###### Country Codes dimention table

In [38]:
with open("I94_SAS_Labels_Descriptions.SAS") as f:
    contents = f.readlines()

In [39]:
country_code = {}
for countries in contents[10:298]:
    pair = countries.split('=')
    code, country = pair[0].strip(), pair[1].strip().strip("'")
    country_code[code] = country

In [40]:
df_country_code = pd.DataFrame(list(country_code.items()), columns=['code', 'country'])
df_country_code.head(5)

,code,country
0,236,AFGHANISTAN
1,101,ALBANIA
2,316,ALGERIA
3,102,ANDORRA
4,324,ANGOLA


In [41]:
output_path="s3a://udacity-datalake-msaied2/Project6_Capstone/country.parquet"
country_spark = spark.createDataFrame(df_country_code)
ef.country_dimension(country_spark, output_path)

Country dimention table writing has been completed!


###### immigration_fact Fact Table

In [42]:
# join country and temperature
output_path="s3a://udacity-datalake-msaied2/Project6_Capstone/country_temperature.parquet"
country_temperature = country_spark.select(["*"])\
            .join(temperature_dim, (country_spark.country == upper(temperature_dim.country)), how='full')\
            .select([country_spark.code, country_spark.country, temperature_dim.temperature_id, temperature_dim.average_temperature, temperature_dim.average_temperature_uncertainty])

country_temperature.write.parquet(output_path, mode="overwrite")
print("Country_Temperature dimention table writing has been completed!")

Country_Temperature dimention table writing has been completed!


In [43]:
country_temperature.toPandas().head()

,code,country,temperature_id,average_temperature,average_temperature_uncertainty
0,151,ARMENIA,NaN,NaN,NaN
1,512,BAHAMAS,NaN,NaN,NaN
2,739,INVALID: DRONNING MAUD LAND (ANTARCTICA-NORWAY),NaN,NaN,NaN
3,373,SOUTH AFRICA,NaN,NaN,NaN
4,914,No Country Code (914),NaN,NaN,NaN


In [44]:
output_path="s3a://udacity-datalake-msaied2/Project6_Capstone/immigration.parquet"
immigration =  ef.immigration_fact(immigration_spark,output_path,spark)

Immigration fact table writing has been completed!


In [45]:
immigration.toPandas().head()

,cicid,i94res,depdate,i94mode,i94port,i94cit,i94addr,airline,fltno,ident,code,temperature_id,migrant_id,status_flag_id,visa_id,state_code,arrdate
0,4084316.0,209.0,20573.0,1.0,HHW,209.0,HI,JL,00782,None,209,None,0,0,0,HI,20566.0
1,4422636.0,582.0,20568.0,1.0,MCA,582.0,TX,*GA,XBLNG,None,None,None,1,1,1,TX,20567.0
2,1195600.0,112.0,20571.0,1.0,OGG,148.0,FL,LH,00464,None,112,None,2,0,0,None,20551.0
3,5291768.0,297.0,20581.0,1.0,LOS,297.0,CA,QR,00739,None,297,None,3,0,3,CA,20572.0
4,985523.0,111.0,20553.0,3.0,CHM,111.0,NY,NaN,LAND,None,111,None,4,4,0,NY,20550.0


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [62]:
migrant = spark.read.parquet("s3a://udacity-datalake-msaied2/Project6_Capstone/migrant.parquet")
status = spark.read.parquet("s3a://udacity-datalake-msaied2/Project6_Capstone/status.parquet")
visa = spark.read.parquet("s3a://udacity-datalake-msaied2/Project6_Capstone/visa.parquet")
state = spark.read.parquet("s3a://udacity-datalake-msaied2/Project6_Capstone/state.parquet")
time = spark.read.parquet("s3a://udacity-datalake-msaied2/Project6_Capstone/time.parquet")
airport = spark.read.parquet("s3a://udacity-datalake-msaied2/Project6_Capstone/airport_codes.parquet")
country = spark.read.parquet("s3a://udacity-datalake-msaied2/Project6_Capstone/country.parquet")
country_temperature = spark.read.parquet("s3a://udacity-datalake-msaied2/Project6_Capstone/country_temperature.parquet")
temperature = spark.read.parquet("s3a://udacity-datalake-msaied2/Project6_Capstone/temperature.parquet")
immigration = spark.read.parquet("s3a://udacity-datalake-msaied2/Project6_Capstone/immigration.parquet")

#### First, check populated tables

In [48]:
tables = {
    "airport": airport,
    "country": country,
    "temperature": temperature,
    "migrant": migrant,
    "state": state,
    "status": status,
    "time": time,
    "visa": visa,
    "country_temperature": country_temperature,
    "immigration": immigration
}

for table_name, table in tables.items():
    record_count = table.count()

    if (record_count == 0):
        print("Data quality check failed for {} with zero records!".format(table_name))
    else:
        print("Data quality check passed for {} with record_count: {} records.".format(table_name, record_count))

Data quality check passed for airport with record_count: 55075 records.
Data quality check passed for country with record_count: 288 records.
Data quality check passed for temperature with record_count: 1 records.
Data quality check passed for migrant with record_count: 215 records.
Data quality check passed for state with record_count: 47 records.
Data quality check passed for status with record_count: 34 records.
Data quality check passed for time with record_count: 30 records.
Data quality check passed for visa with record_count: 149 records.
Data quality check passed for country_temperature with record_count: 289 records.
Data quality check passed for immigration with record_count: 1000 records.


#### Second, Analytics Queries

In [63]:
migrant.createOrReplaceTempView("migrant_tbl")
status.createOrReplaceTempView("status_tbl")
visa.createOrReplaceTempView("visa_tbl")
state.createOrReplaceTempView("state_tbl")
time.createOrReplaceTempView("time_tbl")
airport.createOrReplaceTempView("airport_tbl")
country.createOrReplaceTempView("country_tbl")
country_temperature.createOrReplaceTempView("country_temperature_tbl")
temperature.createOrReplaceTempView("temperature_tbl")
immigration.createOrReplaceTempView("immigration_tbl")

##### Query1: Airport immigration distribution

In [64]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
airport_distribution_df = sqlContext.sql("""
select a.name, a.iso_region, count(i.migrant_id) 
from immigration_tbl i inner join airport_tbl a 
on i.ident = a.ident group by a.name, a.iso_region
""")

In [68]:
airport_distribution_df.toPandas().head()

,name,iso_region,count(migrant_id)
0,San Fernando Airport,US-CA,55
1,Dalnerechensk Airport,RU-PRI,1


##### Query2: US State immigration distribution

In [71]:
state_distribution_df = sqlContext.sql("""
select s.state, count(i.migrant_id) 
from immigration_tbl i inner join state_tbl s 
on i.state_code = s.state_code 
group by s.state
order by count(i.migrant_id) desc
""")

In [72]:
state_distribution_df.toPandas().head()

,state,count(migrant_id)
0,California,163
1,New York,161
2,Hawaii,53
3,Texas,42
4,Nevada,34


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

### Please refer to Data_Dictionary.txt file

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

#### Tools and Technologies
1. AWS S3 for data storage. 
    
    Reason: cheap storage, maintanable by AWS organization (don't worry about scalability), pay as as being used and can be easily accessable and retrieved the data from. 
      
      
2. Pandas for sample data set exploratory data analysis

    Reason: Pandas is very powerful python library for Data Analysis and exploration and can be used easily for schema on read purposes and it's using Data Frames which can be maintained, processed, enriched and changed programmatically. 
    

3. PySpark for large data set data processing to transform staging table to dimensional table

    Reason: PySpark is a handy tool for data scientists since it makes the process of converting prototype models into production-ready model workflows much more effortless. PySpark allows us to create model workflows in cluster environments for model training and serving. Exploratory data analysis (EDA) is crucial in determining data collection structure in a data science workflow, and PySpark can be used for exploratory data analysis and building machine learning pipelines. Another reason to use PySpark is that it has the benefit of being able to scale to far more giant data sets compared to the Python Pandas library.

#### Data Update Frequency
1. All tables should be update in an append-only mode.
2. Tables created from immigration and temperature data set should be updated monthly since the raw data set is built up monthly.
3. Tables created from demography data set could be updated annually since demography data collection takes time and high frequent demography might take high cost but generate wrong conclusion.

#### Considerations/Assumptions:
##### The data was increased by 100x:
We can use more powerful tools for massive data processing and storage like (Amazon EMR & Amazon RedShift)
or use nosql techniques like (Apache Cassandra)

##### The data populates a dashboard that must be updated on a daily basis by 7am every day.
We can use Apache Airflow to hadle the daily scheduling. Also it can be used to apply data quality checks and get some useful notifications.

##### The database needed to be accessed by 100+ people
AWS Redshift can handle a massive amount of concurrent sessions. I believe it's worthy to convert the whole solution to be on cloud based if such number of hits will be there.
